In [1]:
import itertools
import os
import random
import subprocess

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
BASENAME_CSV = 'sorted-20241128_210446' 
I_MAP = 3

DIRECTORY_IMAGES = f'images/{BASENAME_CSV}'
os.makedirs(DIRECTORY_IMAGES, exist_ok=True)    

df_orig = pd.read_csv(f'../logs/rundirs/{BASENAME_CSV}.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,...,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions
0,20241128_191844,map-generator/generated-maps/2024-11-28_13:17:...,0,HumanDrivenVehicle,730.84,6,4375.4,19,0,0,...,NaN,"(773.8, 36.1)",1.2,1.0,731,300,267.9,299.5,DRIVING,"2: [729, 731]"
1,20241128_191844,map-generator/generated-maps/2024-11-28_13:17:...,1,AutonomousVehicle,496.10,7,7721.6,15,0,0,...,NaN,"(437.3, 353.8)",787.6,785.0,990,NaN,959.8,NaN,DRIVING,1: [990]
2,20241128_191844,map-generator/generated-maps/2024-11-28_13:17:...,2,AutonomousVehicle,306.84,11,6921.0,19,0,0,...,NaN,"(440.6, 179.9)",188.9,189.0,612,NaN,581.1,NaN,DRIVING,1: [612]
3,20241128_191844,map-generator/generated-maps/2024-11-28_13:17:...,3,AutonomousVehicle,441.44,9,8806.8,12,0,0,...,NaN,"(588.0, 464.0)",876.9,874.0,880,NaN,850.2,NaN,DRIVING,1: [880]
4,20241128_192026,map-generator/generated-maps/2024-11-28_13:17:...,0,HumanDrivenVehicle,730.84,12,8999.7,12,11,11,...,NaN,"(553.1, 109.4)",250.5,250.0,731,301,269.0,NaN,DRIVING,"2: [729, 731]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,20241128_204909,map-generator/generated-maps/2024-11-28_13:19:...,3,AutonomousVehicle,781.52,6,9397.2,7,0,0,...,NaN,"(736.7, 509.3)",27.7,27.0,1560,562,530.4,NaN,DRIVING,1: [1560]
236,20241128_205101,map-generator/generated-maps/2024-11-28_13:19:...,0,HumanDrivenVehicle,732.00,12,8916.4,13,11,11,...,NaN,"(636.9, 71.1)",151.8,151.0,732,296,264.5,145.6,DRIVING,"2: [730, 732]"
237,20241128_205101,map-generator/generated-maps/2024-11-28_13:19:...,1,AutonomousVehicle,430.25,9,8036.4,19,0,0,...,NaN,"(427.0, 177.9)",306.6,306.0,858,381,349.8,77.1,DRIVING,1: [858]
238,20241128_205101,map-generator/generated-maps/2024-11-28_13:19:...,2,AutonomousVehicle,453.00,9,8592.8,15,0,0,...,NaN,"(489.0, 85.1)",454.1,453.0,904,NaN,873.2,NaN,DRIVING,1: [904]


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id,
    df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/(?P<basename_scenario>[^/]+)[.]json$', expand=True),
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+)[.]json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id['filename_screenshot'] = "../map-generator/generated-maps/" + df_id['dir_map'] + '/screenshots/' + df_id['basename_scenario'] + '.png'
df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
df_id['configuration'] = df_id[
    ['i_map', 'are_bridges', 'Positions variant']].agg(
    lambda r: f'map {r['i_map']}, {"with" if r['are_bridges'] else "without"} bridges, pos.var. {r['Positions variant']}', 
    axis=1
)
df_id = df_id.sort_values(['i_map', 'are_bridges', 'Positions variant'], ascending=[True, False, True])
df_id = df_id[df_id['i_map'] == I_MAP]
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,dir_map,basename_scenario,i_map,Positions variant,probabilityForcingForHuman,filename_screenshot,are_bridges,configuration
0,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 3, with bridges, pos.var. 1"
1,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 3, with bridges, pos.var. 1"
2,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 3, with bridges, pos.var. 1"
3,map-generator/generated-maps/2024-11-28_13:17:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 3, with bridges, pos.var. 1"
4,map-generator/generated-maps/2024-11-28_13:17:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:17:39_with_bridges,scenario3-1,3,1,1.0,../map-generator/generated-maps/2024-11-28_13:...,True,"map 3, with bridges, pos.var. 1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,map-generator/generated-maps/2024-11-28_13:19:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 3, without bridges, pos.var. 10"
140,map-generator/generated-maps/2024-11-28_13:19:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 3, without bridges, pos.var. 10"
141,map-generator/generated-maps/2024-11-28_13:19:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 3, without bridges, pos.var. 10"
142,map-generator/generated-maps/2024-11-28_13:19:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-28_13:19:18_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-28_13:...,False,"map 3, without bridges, pos.var. 10"


In [4]:
configuration_to_filename_screenshot = {row['configuration']: row['filename_screenshot'] for _, row in df_id.iterrows()}
configuration_to_filename_screenshot

{'map 3, with bridges, pos.var. 1': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-1.png',
 'map 3, with bridges, pos.var. 2': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-2.png',
 'map 3, with bridges, pos.var. 3': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-3.png',
 'map 3, with bridges, pos.var. 4': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-4.png',
 'map 3, with bridges, pos.var. 5': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-5.png',
 'map 3, with bridges, pos.var. 6': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-6.png',
 'map 3, with bridges, pos.var. 7': '../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario3-7.png',
 'map 3, with bridges, pos.var. 8': '../map-generator/generated-maps/2024-11

In [5]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([
    df_id[['i_map', 'are_bridges', 'Positions variant', 'configuration', 
           'Coordination strategy', 'probabilityForcingForHuman']], 
    df_orig.iloc[:, index_id + 1:]], 
    axis=1
)
df

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,...,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions
0,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,0,HumanDrivenVehicle,730.84,6,...,NaN,"(773.8, 36.1)",1.2,1.0,731,300,267.9,299.5,DRIVING,"2: [729, 731]"
1,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,1,AutonomousVehicle,496.10,7,...,NaN,"(437.3, 353.8)",787.6,785.0,990,NaN,959.8,NaN,DRIVING,1: [990]
2,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,2,AutonomousVehicle,306.84,11,...,NaN,"(440.6, 179.9)",188.9,189.0,612,NaN,581.1,NaN,DRIVING,1: [612]
3,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,3,AutonomousVehicle,441.44,9,...,NaN,"(588.0, 464.0)",876.9,874.0,880,NaN,850.2,NaN,DRIVING,1: [880]
4,3,True,1,"map 3, with bridges, pos.var. 1",change of priorities,1.0,0,HumanDrivenVehicle,730.84,12,...,NaN,"(553.1, 109.4)",250.5,250.0,731,301,269.0,NaN,DRIVING,"2: [729, 731]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3,False,10,"map 3, without bridges, pos.var. 10",change of priorities,1.0,3,AutonomousVehicle,781.52,6,...,NaN,"(735.3, 379.6)",157.4,157.0,1560,673,642.1,NaN,DRIVING,1: [1560]
140,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,0,HumanDrivenVehicle,732.00,12,...,NaN,"(559.0, 108.2)",244.9,244.0,732,426,395.2,NaN,DRIVING,"2: [730, 732]"
141,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,1,AutonomousVehicle,496.98,5,...,NaN,"(441.8, 223.7)",659.8,657.0,992,NaN,961.6,NaN,DRIVING,1: [992]
142,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,2,AutonomousVehicle,477.96,8,...,NaN,"(537.2, 448.3)",867.2,865.0,954,NaN,923.3,NaN,DRIVING,1: [954]


In [6]:
series_blocks = df[df['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['configuration', 'are_bridges'], sort=False)['Is blocked'].sum()
index_blocked = series_blocks[series_blocks != 0].index
index_nonblocked = series_blocks[series_blocks == 0].index
index_nonblocked[~index_nonblocked.get_level_values('are_bridges')].get_level_values('configuration')

Index(['map 3, without bridges, pos.var. 1',
       'map 3, without bridges, pos.var. 2',
       'map 3, without bridges, pos.var. 3',
       'map 3, without bridges, pos.var. 4',
       'map 3, without bridges, pos.var. 5',
       'map 3, without bridges, pos.var. 6',
       'map 3, without bridges, pos.var. 7',
       'map 3, without bridges, pos.var. 8',
       'map 3, without bridges, pos.var. 9',
       'map 3, without bridges, pos.var. 10'],
      dtype='object', name='configuration')

In [9]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_IMAGES}/{basename}.png'
    fig.savefig(filename)
    #fig.close()
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    return filename


def same_value(series):
    assert series.nunique() == 1, series
    return series.iloc[0]
    

def make_misbehaviors(df_nonbaseline):
    misbehaviors = []
    
    probabilityForcingForHuman = same_value(df_nonbaseline['probabilityForcingForHuman'])
    if probabilityForcingForHuman > 0.0:
        misbehaviors.append(f'violation of priorities ({"random" if probabilityForcingForHuman < 1.0 else "constant"})')
        
    isCanPassFirstActive = same_value(df_nonbaseline['isCanPassFirstActive'])
    if isCanPassFirstActive.startswith('hum=true, '):
        misbehaviors.append('can pass first')
    elif isCanPassFirstActive.startswith('hum=false, '):
        pass
    else:
        raise ValueError(isCanPassFirstActive)
    
    if 'probabilitySlowingDownForHuman' in df_nonbaseline.columns:
        probabilitySlowingDownForHuman = float(same_value(df_nonbaseline['probabilitySlowingDownForHuman']))
        if probabilitySlowingDownForHuman > 0.0:
            misbehaviors.append(f'moving slowly ({"random" if probabilitySlowingDownForHuman < 1.0 else "constant"})')
    
    return misbehaviors
    
    
class Formula:
    def __init__(self, label, expression=None):
        self.label = label
        self.expression = expression if expression is not None else label
        
    def __str__(self):
        return self.label.replace('`', '')
    
    def __repr__(self):
        return f'<Formula: {self.label!r}>'
    
    def apply(self, df):
        if isinstance(self.expression, str):
            return df.eval(self.expression)
        return df.apply(self.expression, axis=1)
    

def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, mode):
    #IPython.display.display(df)
    
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    strategy2label = {'baseline': 'baseline\n(no human effect)', 'stops': 'stops (local)'}        
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    
    formulas_y2_aut = [Formula('`No. of collisions`'), Formula('`No. of near-misses`')]
    colors_y2_aut = ['red', 'yellow']
    
    formulas_y2_hum = [Formula('`No. of violations`')] + formulas_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    formulas_y2_cmp = [Formula(f'`{col}` / `No. of violations`', 
                               lambda row, col=col: row[col] / row['No. of violations'] if row['No. of violations'] > 0 else 0)
                       for col in ('No. of collisions', 'No. of near-misses')]
    colors_y2_cmp = ['red', 'yellow']
    
    if mode in ('aut', 'hum'):
        column_y1 = 'Total distance traveled (m)'
        color_y1 = 'tab:blue'
        
        if mode == 'aut':
            formulas_y2 = formulas_y2_aut
            colors_y2 = colors_y2_aut
            title1 = 'Automated vehicles (summarised for AV1-AV3)' 
        else:
            formulas_y2 = formulas_y2_hum
            colors_y2 = colors_y2_hum
            title1 = 'Human-driven vehicle'
            
    elif mode == 'cmp':
        column_y1 = None
        color_y1 = None
        
        formulas_y2 = formulas_y2_cmp
        colors_y2 = colors_y2_cmp
        title1 = 'Collisions rate'
        
    else:
        raise ValueError(mode)
    
    # Get the global min and max values for consistent y-axis scaling
    y1_max = None if column_y1 is None else max(dfx[column_y1].max() for dfx in dfs_y1)
    y2_maxes = [formula.apply(dfx).max()
                for dfx in dfs_y2
                for formula in (formulas_y2 if mode == 'cmp' else set(formulas_y2_aut + formulas_y2_hum))]
    y2_max = max(y2_maxes)
    #print(y2_max, formulas_y2, y2_maxes)
    #print(f'{dfs_y2=}')
    
    # Iterate through each Positions variant
    for i, position in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[position]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars
        handles = []
        if column_y1 is not None:
            handles += [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
            ax.set_xlabel('Coordination Strategy')
            ax.set_ylabel(column_y1, color=color_y1)
            ax.tick_params(axis='y', labelcolor=color_y1)
            ax.set_ylim(0, y1_max * 1.1)
        else:
            ax.set_axis_off()
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points
        handles += [ax_right.plot(x_positions, formula.apply(df_pos), label=str(formula),
                                  marker='o', linestyle='', color=color)[0]
                    for formula, color in zip(formulas_y2, colors_y2)]
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels([strategy2label.get(s, s) for s in strategies], rotation=45, ha='right')
        ax.set_title(f'Position {position}')
        ax.grid(axis='y')
        
    misbehaviors = make_misbehaviors(df[df['Coordination strategy'] != 'baseline'])
    title3 = 'Human (mis)behaviour actions: ' + ('none' if not misbehaviors else ', '.join(misbehaviors))    
    
    title = title1
    if mode == 'aut':
        title = f'{title2}\n{title3}\n\n{title}'
    fig.suptitle(title, fontsize=16)
    
    labels = [str(x) for x in [column_y1, *formulas_y2] if x is not None]
    fig.legend(handles=handles, labels=labels, ncol=len(handles), loc='upper right')
    
    fig.tight_layout()
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    plt.close()
    
    #print(id(df))
    #IPython.display.display(df)
    
    return filename_png


key2df = {}
for are_bridges in True, False:
    for is_aut in True, False:
        dfx = df[df['are_bridges'] == are_bridges]
        dfx = dfx[dfx['configuration'].isin(index_nonblocked.get_level_values('configuration'))]
        dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
        dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).agg({
            **{col: 'sum' for col in dfx.columns},
            **{col: same_value for col in (
                'Positions variant', 'Coordination strategy', 'configuration', 
                'probabilityForcingForHuman', 'isCanPassFirstActive', 'probabilitySlowingDownForHuman',
            ) if col in dfx.columns}
        })
        key2df[are_bridges, is_aut] = dfx    
        
for are_bridges in True, False:
    filenames_png = []
    for mode in 'aut', 'hum', 'cmp':
        is_aut = mode == 'aut'
        try:
            filename_png = plot_aut_hum(
                key2df[are_bridges, is_aut],
                title2=f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)", 
                dfs_y1=[dfx for (_, is_aut_dfx), dfx in key2df.items() if is_aut_dfx == is_aut], 
                dfs_y2=key2df.values() if mode != 'cmp' else [key2df[are_bridges, is_aut]], 
                mode=mode,
            )
            filenames_png.append(filename_png)
        except Exception as exc:
            #print(type(exc), exc)
            raise exc
    #IPython.display.display(dfx)
    filename_out_png = f'{DIRECTORY_IMAGES}/All: Map {I_MAP} (' + ('with' if are_bridges else 'without') + ' bridges).png'
    subprocess.run(['convert', *filenames_png, '-append', filename_out_png], check=True)
    print()

In [11]:
def show_maps(title, configurations, ncols): 
    nrows = max(1, (len(configurations) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(configurations)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, configuration in zip(axes, configurations):
        filename_screenshot = configuration_to_filename_screenshot[configuration]
        try:
            image = plt.imread(filename_screenshot)
            ax.imshow(image)
        except Exception as exc:
            print(exc)
        ax.title.set_text(f'Configuration:\n{configuration}')
    
    fig.suptitle(title, fontsize=16)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    save_and_show(fig, title)
    plt.close()
    
    
for title, index in {'Non-blocked': index_nonblocked, 'Blocked': index_blocked}.items():
    for are_bridges in True, False:
        show_maps(f'{title}: Map {I_MAP} ({"with" if are_bridges else "without"} bridges)', 
                  index[index.get_level_values('are_bridges') == are_bridges].get_level_values('configuration'), 
                  4)